In [1]:
import pandas as pd
import numpy as np

import pickle

# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import nltk
import spacy

from textblob import TextBlob, Word

import re

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pd.set_option('display.max_colwidth', 400)  # or 199
pd.set_option('display.max_columns', None)  


In [3]:
tweet_stopwords = stopwords.words('english') + \
    ['rt', 'https', 'http', 'amp', 'via', 'one', 'around', 'would', 'let', 'could', 'going', 'like', 
     'get', 'may', 'says', 'say', 'make', 'based', 'even', 'another', 'completely', 'thanks', 'way', 
     'find', 'used', 'thing', '2019', 'see', 'need', 'know', 'knows', 'think', 'thinks', 'take', 'new', 
     'day', 'days', 'aoc', 'alexandria', 'ocasio', 'cortez', 'ocasio-cortez', 'pron']

In [4]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [5]:
# display_topics(lsa_0, tfidf_0.get_feature_names(), 10)

OK, so now I need to append the topic for each document to a pandas dataframe, along with topic scores, to see if this is owrking as expected. If it is, I can make visualizations and move on to sentiment score

w00t, did it

append topic to community df, look at top tweets per topic, sentiment analysis on those

In [6]:
## Importing DFs:
## - 2 (celebs), 6 (AOC), 7 (Bill Nye, journalists)
## - 4 (charlie kirk), 5 (shapiro), 9 (foxnews, hannity)

with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/raw_dfs_by_com/df_2.pickle', 'rb') as f:
    df_com_2 = pickle.load(f)
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/raw_dfs_by_com/df_4.pickle', 'rb') as f:
    df_com_4 = pickle.load(f)
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/raw_dfs_by_com/df_5.pickle', 'rb') as f:
    df_com_5 = pickle.load(f)
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/raw_dfs_by_com/df_6.pickle', 'rb') as f:
    df_com_6 = pickle.load(f)
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/raw_dfs_by_com/df_7.pickle', 'rb') as f:
    df_com_7 = pickle.load(f)
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/raw_dfs_by_com/df_9.pickle', 'rb') as f:
    df_com_9 = pickle.load(f)
    

In [7]:
## Importing preprocessed text for same communities as above

with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_2_preprocess.pickle', 'rb') as f:
    df_2 = pickle.load(f)
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_4_preprocess.pickle', 'rb') as f:
    df_4 = pickle.load(f)
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_5_preprocess.pickle', 'rb') as f:
    df_5 = pickle.load(f)
    
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_6_preprocess.pickle', 'rb') as f:
    df_6 = pickle.load(f)
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_7_preprocess.pickle', 'rb') as f:
    df_7 = pickle.load(f)
    
with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_9_preprocess.pickle', 'rb') as f:
    df_9 = pickle.load(f)

In [8]:
tfidf_2 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_4 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_5 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_6 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_7 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_9 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))

In [9]:
X_2 = df_2['tweet_processed']
X_4 = df_4['tweet_processed']
X_5 = df_5['tweet_processed']
X_6 = df_6['tweet_processed']
X_7 = df_7['tweet_processed']
X_9 = df_9['tweet_processed']

In [10]:
bag_of_words_2 = tfidf_2.fit_transform(X_2)
bag_of_words_4 = tfidf_4.fit_transform(X_4)
bag_of_words_5 = tfidf_5.fit_transform(X_5)
bag_of_words_6 = tfidf_6.fit_transform(X_6)
bag_of_words_7 = tfidf_7.fit_transform(X_7)
bag_of_words_9 = tfidf_9.fit_transform(X_9)

In [11]:
nmf_2 = NMF(20)
doc_topic_NMF_2 = nmf_2.fit_transform(bag_of_words_2)

nmf_4 = NMF(20)
doc_topic_NMF_4 = nmf_4.fit_transform(bag_of_words_4)

nmf_5 = NMF(20)
doc_topic_NMF_5 = nmf_5.fit_transform(bag_of_words_5)

nmf_6 = NMF(20)
doc_topic_NMF_6 = nmf_6.fit_transform(bag_of_words_6)

nmf_7 = NMF(20)
doc_topic_NMF_7 = nmf_7.fit_transform(bag_of_words_7)

nmf_9 = NMF(20)
doc_topic_NMF_9 = nmf_9.fit_transform(bag_of_words_9)

In [12]:
nmf_com_2_df = pd.DataFrame(data=doc_topic_NMF_2)
nmf_com_4_df = pd.DataFrame(data=doc_topic_NMF_4)
nmf_com_5_df = pd.DataFrame(data=doc_topic_NMF_5)
nmf_com_6_df = pd.DataFrame(data=doc_topic_NMF_6)
nmf_com_7_df = pd.DataFrame(data=doc_topic_NMF_7)
nmf_com_9_df = pd.DataFrame(data=doc_topic_NMF_9)

In [13]:
nmf_com_2_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.109369,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.000000,0.0,0.3608,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.332341,0.0,0.0000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.433731,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.470663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
nmf_com_2_df['doc_topic'] =  nmf_com_2_df.idxmax(axis=1)
nmf_com_2_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,doc_topic
0,0.109369,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.000000,0.0,0.0,0.000000,0.0,0.3608,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,0.000000,0.0,0.0,0.332341,0.0,0.0000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.433731,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.470663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [15]:
nmf_com_4_df['doc_topic'] =  nmf_com_4_df.idxmax(axis=1)
nmf_com_5_df['doc_topic'] =  nmf_com_5_df.idxmax(axis=1)
nmf_com_6_df['doc_topic'] =  nmf_com_6_df.idxmax(axis=1)
nmf_com_7_df['doc_topic'] =  nmf_com_7_df.idxmax(axis=1)
nmf_com_9_df['doc_topic'] =  nmf_com_9_df.idxmax(axis=1)


In [16]:
nmf_com_4_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,doc_topic
0,0.105189,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0
1,0.000000,0.11706,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1
2,0.000000,0.00000,0.000000,0.124018,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,3
3,0.000000,0.00000,0.000000,0.000000,0.124106,0.000000,0.0,0.000000,0.000000,0.000000,0.000090,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,4
4,0.001217,0.00356,0.003669,0.000000,0.000000,0.000211,0.0,0.000603,0.000035,0.000165,0.000278,0.0,0.0,0.000574,0.0,0.023004,0.0,0.000024,0.0,0.0,15


Once above is done, can then append topic assignment to each community df, then find tweets by topic

In [17]:
df_com_2 = df_com_2.reset_index()
df_com_4 = df_com_4.reset_index()
df_com_5 = df_com_5.reset_index()
df_com_6 = df_com_6.reset_index()
df_com_7 = df_com_7.reset_index()
df_com_9 = df_com_9.reset_index()
df_com_2.head()

,index,screen_name,followers_count,modularity_class,main_text,retweet_text,tweet_date,PageRank,rt_count
0,788552,MarkRuffalo,4631012,2.0,"I was a bartender for a long, long time. Everyone should be required to work in service or customer service jobs. Lots of life lessons learned. https://t.co/7FzVfdsnuE",NaN,Sat Mar 09 22:35:49 +0000 2019,0.012844,13505
1,985677,MarkRuffalo,4631154,2.0,More Thin Mints for the rest of us 🤷🏻‍♂️ https://t.co/POvmEMDNLZ,NaN,Fri Mar 08 23:14:38 +0000 2019,0.012844,185
2,816346,DaveedDiggs,385097,2.0,That part https://t.co/GDjSH202dk,NaN,Sat Mar 09 19:12:16 +0000 2019,0.000134,118
3,681000,voguemagazine,13585706,2.0,"""People want to work for what they get,"" said Ivanka Trump, famous heiress whose career has depended almost entirely on being given jobs by her father. https://t.co/MTvSzv0Sco",NaN,Sun Mar 10 06:54:55 +0000 2019,0.000043,54
4,1071225,StaceyDooley,278741,2.0,FFS. THIS WOMAN. I CANNNNN’T. \n😩👊🏽❤️🔥✨💪🏽🥳😻💅🏼🙌🏼 https://t.co/l62GcV5p1C,NaN,Fri Mar 08 17:40:53 +0000 2019,0.000036,24


In [18]:
# display_topics(nmf_2, tfidf_2.get_feature_names(), 10)

In [19]:
df_com_2 = pd.merge(df_com_2, nmf_com_2_df, left_index=True, right_index=True)

In [20]:
df_com_4 = pd.merge(df_com_4, nmf_com_4_df, left_index=True, right_index=True)
df_com_5 = pd.merge(df_com_5, nmf_com_5_df, left_index=True, right_index=True)
df_com_6 = pd.merge(df_com_6, nmf_com_6_df, left_index=True, right_index=True)
df_com_7 = pd.merge(df_com_7, nmf_com_7_df, left_index=True, right_index=True)
df_com_9 = pd.merge(df_com_9, nmf_com_9_df, left_index=True, right_index=True)

In [42]:
df_com_4.sample(10)

,index,screen_name,followers_count,modularity_class,main_text,retweet_text,tweet_date,PageRank,rt_count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,doc_topic,polarity,subjectivity,pol_z
125160,1329600,aaronwood138,316,4.0,"RT @DemocratsLogic: Don’t worry, @aoc knows who committed the FEC violations she’s been accused of. https://t.co/pRMT1mdSYg","Don’t worry, @aoc knows who committed the FEC violations she’s been accused of. https://t.co/pRMT1mdSYg",Thu Mar 07 20:59:26 +0000 2019,1.893602e-06,24,0.000000,0.000000,0.000311,0.000000,0.000000,0.000588,0.001785,0.004779,0.000211,0.000514,0.000376,0.000000,0.000047,0.000545,0.000294,0.000573,0.000316,0.000530,0.000000,0.000338,7,0.000000,0.00,0.487568
12312,277716,YankeeRunnerLV,837,4.0,"RT @RealWayneRoot: Put moron Alexandria Ocasio Cortez in charge. Soon, this will be America. Got guns? Got gold? Got survival food? https:/…","Put moron Alexandria Ocasio Cortez in charge. Soon, this will be America. Got guns? Got gold? Got survival food? https://t.co/Lzo5Hd2N6Y",Mon Mar 11 13:52:49 +0000 2019,1.116091e-05,56,0.000000,0.000000,0.000242,0.001199,0.000016,0.000438,0.000000,0.000559,0.000205,0.000415,0.000335,0.000000,0.000073,0.000404,0.000115,0.003921,0.000262,0.000328,0.000000,0.000219,15,-0.800000,1.00,-2.790532
195689,1424173,Sat_102,34,4.0,RT @charliekirk11: Does @AOC actually live in the district she claims to represent?\n\nNeighbors say they've never seen her or her boyfriend…,"Does @AOC actually live in the district she claims to represent?\n\nNeighbors say they've never seen her or her boyfriend at ""their"" apartment\n\nLocal businesses claim she only visits to be filmed by news crews\n\nPostal workers claim mail piles up for months in her box\n\nFraud?\n\n🤔",Thu Mar 07 13:58:48 +0000 2019,8.440498e-07,19196,0.105189,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.136364,0.50,0.198124
58542,1187573,SWysel07,66,4.0,"@WayneDupreeShow Of course, @aoc is the boss, after all. Our laws don't apply to her.",NaN,Fri Mar 08 06:47:50 +0000 2019,4.194710e-06,0,0.000000,0.000026,0.000016,0.000000,0.000015,0.000020,0.000057,0.000099,0.000061,0.000039,0.000055,0.000006,0.000006,0.000040,0.000017,0.000123,0.000165,0.000064,0.000063,0.000146,16,0.000000,0.00,0.155208
66584,424891,HoneyBadger0603,82,4.0,RT @neontaster: AOC blasts capitalism as irredeemable at an event that generates $350 million a year for the local economy. https://t.co/kO…,AOC blasts capitalism as irredeemable at an event that generates $350 million a year for the local economy. https://t.co/kOcp2saEGL,Mon Mar 11 00:19:39 +0000 2019,3.785948e-06,1904,0.001675,0.000000,0.000211,0.000000,0.001622,0.000536,0.001672,0.000499,0.002392,0.000522,0.000065,0.000000,0.000105,0.000362,0.003089,0.000593,0.000000,0.000490,0.000000,0.004329,19,0.000000,0.00,0.008721
122025,215405,Pamala31969,3433,4.0,"RT @1776Stonewall: AOC, attacking America, says that we've created an environment that ""if you do not have a job you are left to die"". Firs…","AOC, attacking America, says that we've created an environment that ""if you do not have a job you are left to die"". First of all, She literally just described to Venezuela. And not having a job? That's the Democrats entire platform, government handouts",Mon Mar 11 17:04:23 +0000 2019,1.945770e-06,357,0.000000,0.001022,0.000000,0.003156,0.000204,0.000011,0.000000,0.000000,0.000993,0.003242,0.000000,0.000145,0.000008,0.000115,0.000000,0.004473,0.001915,0.000039,0.002308,0.000111,15,0.000000,0.00,-0.305768
57858,1059053,Ironheaddugg,2321,4.0,RT @benshapiro: So AOC runs cover for an anti-Semite and then accuses those who oppose anti-Semitism of anti-Semitism. And now the media wi…,So AOC runs cover for an anti-Semite and then accuses those who oppose anti-Semitism of anti-Semitism. And now the media will echo her. Gaslighting is fun!,Fri Mar 08 18:19:17 +0000 2019,4.2

So now I should do a groupby to find the most popular 5 topics in each community, and find one that I can hone in on for sentiment analysis (one that is present in each ideally, if not, then one that is present in at least 2)

In [22]:
# df_com_2.sample(10)

In [23]:
top_topics_2 = df_com_2.groupby(['doc_topic'])[['index']].count().reset_index().nlargest(5, 'index')
top_topics_4 = df_com_4.groupby(['doc_topic'])[['index']].count().reset_index().nlargest(5, 'index')
top_topics_5 = df_com_5.groupby(['doc_topic'])[['index']].count().reset_index().nlargest(5, 'index')
top_topics_6 = df_com_6.groupby(['doc_topic'])[['index']].count().reset_index().nlargest(5, 'index')
top_topics_7 = df_com_7.groupby(['doc_topic'])[['index']].count().reset_index().nlargest(5, 'index')
top_topics_9 = df_com_9.groupby(['doc_topic'])[['index']].count().reset_index().nlargest(5, 'index')


In [30]:
top_topics_2

,doc_topic,index
0,0,6994
19,19,349
1,1,155
2,2,108
3,3,84


use textblob to calculate document polarity and subjectivitym, and pol_z to see how close or far the tweet is from it's community

In [31]:
from textblob import TextBlob


First, need to grab the text I need:

In [43]:
def choose_text(df):
    # selects the full text version of the tweet for analysis
    df['final_text'] = np.where((df['retweet_text'].isnull() == False), df['retweet_text'], df['main_text'])
    return df

In [46]:
df_com_2 = choose_text(df_com_2)
df_com_4 = choose_text(df_com_4)
df_com_5 = choose_text(df_com_5)
df_com_6 = choose_text(df_com_6)
df_com_7 = choose_text(df_com_7)
df_com_9 = choose_text(df_com_9)

In [47]:
df_com_2['polarity'] = df_com_2.final_text.apply(lambda x: TextBlob(x).sentiment.polarity)
df_com_2['subjectivity'] = df_com_2.final_text.apply(lambda x: TextBlob(x).sentiment.subjectivity)
df_com_2['pol_z'] = df_com_2.groupby(['modularity_class','doc_topic']).polarity.transform(lambda x: (x-np.mean(x))/np.std(x))

In [48]:
df_com_4['polarity'] = df_com_4.final_text.apply(lambda x: TextBlob(x).sentiment.polarity)
df_com_4['subjectivity'] = df_com_4.final_text.apply(lambda x: TextBlob(x).sentiment.subjectivity)
df_com_4['pol_z'] = df_com_4.groupby(['modularity_class','doc_topic']).polarity.transform(lambda x: (x-np.mean(x))/np.std(x))

In [49]:
df_com_5['polarity'] = df_com_5.final_text.apply(lambda x: TextBlob(x).sentiment.polarity)
df_com_5['subjectivity'] = df_com_5.final_text.apply(lambda x: TextBlob(x).sentiment.subjectivity)
df_com_5['pol_z'] = df_com_5.groupby(['modularity_class','doc_topic']).polarity.transform(lambda x: (x-np.mean(x))/np.std(x))

In [50]:
df_com_6['polarity'] = df_com_6.main_text.apply(lambda x: TextBlob(x).sentiment.polarity)
df_com_6['subjectivity'] = df_com_6.main_text.apply(lambda x: TextBlob(x).sentiment.subjectivity)
df_com_6['pol_z'] = df_com_6.groupby(['modularity_class','doc_topic']).polarity.transform(lambda x: (x-np.mean(x))/np.std(x))

In [51]:
df_com_7['polarity'] = df_com_7.final_text.apply(lambda x: TextBlob(x).sentiment.polarity)
df_com_7['subjectivity'] = df_com_7.final_text.apply(lambda x: TextBlob(x).sentiment.subjectivity)
df_com_7['pol_z'] = df_com_7.groupby(['modularity_class','doc_topic']).polarity.transform(lambda x: (x-np.mean(x))/np.std(x))

In [52]:
df_com_9['polarity'] = df_com_9.final_text.apply(lambda x: TextBlob(x).sentiment.polarity)
df_com_9['subjectivity'] = df_com_9.final_text.apply(lambda x: TextBlob(x).sentiment.subjectivity)
df_com_9['pol_z'] = df_com_9.groupby(['modularity_class','doc_topic']).polarity.transform(lambda x: (x-np.mean(x))/np.std(x))

In [53]:
with open('/Users/robertpagano/metis_data/project_5/sentiment_data/df_com_2_sentiment.pickle', 'wb') as to_write:
    pickle.dump(df_com_2, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/sentiment_data/df_com_4_sentiment.pickle', 'wb') as to_write:
    pickle.dump(df_com_4, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/sentiment_data/df_com_5_sentiment.pickle', 'wb') as to_write:
    pickle.dump(df_com_5, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/sentiment_data/df_com_6_sentiment.pickle', 'wb') as to_write:
    pickle.dump(df_com_6, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/sentiment_data/df_com_7_sentiment.pickle', 'wb') as to_write:
    pickle.dump(df_com_7, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/sentiment_data/df_com_9_sentiment.pickle', 'wb') as to_write:
    pickle.dump(df_com_9, to_write)
    


Looking at top topics for 6 (aoc, krassensteins)

In [54]:
top_topics_6

,doc_topic,index
18,18,18157
12,12,14863
14,14,14198
1,1,13819
9,9,12581


In [73]:
# display_topics(nmf_6, tfidf_6.get_feature_names(), 10)

show tweets from top influencers in topic 18 for community 6

In [57]:
top_tweets_com_6_top_18 = df_com_6[df_com_6['doc_topic'] == 18]

In [60]:
# top_tweets_com_6_top_18.head(10)

In [62]:
# top_tweets_com_6_top_12 = df_com_6[df_com_6['doc_topic'] == 12]
# top_tweets_com_6_top_12.head()

In [64]:
# top_tweets_com_6_top_14 = df_com_6[df_com_6['doc_topic'] == 14]
# top_tweets_com_6_top_14.head()

In [81]:
top_topics_9

,doc_topic,index
17,17,87220
19,19,51302
6,6,34402
15,15,33195
13,13,21810


In [85]:
top_tweets_com_9_top_9 = df_com_9[df_com_9['doc_topic'] == 9]
top_tweets_com_9_top_9.head(10)

,index,screen_name,followers_count,modularity_class,main_text,retweet_text,tweet_date,PageRank,rt_count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,doc_topic,polarity,subjectivity,pol_z,final_text
10,109296,RyanAFournier,643596,9.0,RT @TaxReformExpert: Ocasio-Cortez actually said that Capitalism is irredeemable? Really? Oh man. And Nancy Pelosi said NOTHING in respo…,Ocasio-Cortez actually said that Capitalism is irredeemable? Really? Oh man. And Nancy Pelosi said NOTHING in response?,Mon Mar 11 22:51:39 +0000 2019,0.003481,891,0.002474,0.000000,0.000000,0.000000,0.002753,0.002438,0.000000,0.000000,0.000055,0.003805,0.003277,0.000488,0.003135,0.000089,0.000000,0.000408,0.000000,0.002587,0.000113,0.001449,9,0.100000,0.150000,0.183164,Ocasio-Cortez actually said that Capitalism is irredeemable? Really? Oh man. And Nancy Pelosi said NOTHING in response?
41,179532,seanhannity,3984362,9.0,"SERIOUSLY!?: ""We live in a society where if you don’t have a job, you are left to die..."" https://t.co/D63Fxv4NVc",NaN,Mon Mar 11 19:10:05 +0000 2019,0.001968,1127,0.002908,0.000000,0.002531,0.000610,0.000058,0.000123,0.000375,0.000350,0.000000,0.006452,0.000261,0.000515,0.000000,0.000068,0.000522,0.001975,0.000169,0.002312,0.002846,0.000374,9,0.068182,0.250000,0.047999,"SERIOUSLY!?: ""We live in a society where if you don’t have a job, you are left to die..."" https://t.co/D63Fxv4NVc"
60,1519903,hrtablaze,85822,9.0,"Nancy Pelosi got punked by a bartender with a 430 credit score ! 😂\n\nCongratulations AOC , you were right , you are the boss now and you're in charge !",NaN,Thu Mar 07 01:17:31 +0000 2019,0.001965,89,0.000000,0.000086,0.000000,0.000039,0.000054,0.000000,0.000102,0.000000,0.001506,0.003361,0.002421,0.000110,0.000000,0.000364,0.000000,0.000676,0.000000,0.000199,0.000000,0.000357,9,0.357143,0.535714,1.275515,"Nancy Pelosi got punked by a bartender with a 430 credit score ! 😂\n\nCongratulations AOC , you were right , you are the boss now and you're in charge !"
112,788134,SuperEliteTexan,68729,9.0,"RT @LadyRedWave: Newt is SO RIGHT! Beware you OLDSTERS the ""kids"" think THEY R IN charge! @SpeakerPelosi YOU ARE DONE! B'bye! \n#NewtGingric…","Newt is SO RIGHT! Beware you OLDSTERS the ""kids"" think THEY R IN charge! @SpeakerPelosi YOU ARE DONE! B'bye! \n#NewtGingrich #Pelosi’s grip slips as #AOC &amp; friends throw a party, leave the grandparents at home https://t.co/JE2v9nahUv",Sat Mar 09 22:38:01 +0000 2019,0.001826,337,0.000000,0.000000,0.000000,0.000021,0.000193,0.000000,0.000000,0.000055,0.000731,0.070297,0.000000,0.000021,0.000000,0.000000,0.000027,0.000456,0.000000,0.000000,0.000000,0.000000,9,0.697545,0.535714,2.721554,"Newt is SO RIGHT! Beware you OLDSTERS the ""kids"" think THEY R IN charge! @SpeakerPelosi YOU ARE DONE! B'bye! \n#NewtGingrich #Pelosi’s grip slips as #AOC &amp; friends throw a party, leave the grandparents at home https://t.co/JE2v9nahUv"
147,717994,SuperEliteTexan,68729,9.0,"RT @jnottah: Newt Gingrich: Pelosi’s grip slips as Ocasio-Cortez and friends throw a party, leave the grandparents at home https://t.co/UV5…","Newt Gingrich: Pelosi’s grip slips as Ocasio-Cortez and friends throw a party, leave the grandparents at home https://t.co/UV5AAYz5iP #FoxNews For those who vote Dem, You better get use to this, Socialism now, Communism in the near future. Bureaucrats and Slaves #WalkAway",Sun Mar 10 03:25:11 +0000 2019,0.001826,142,0.000000,0.000133,0.000000,0.000128,0.000004,0.001261,0.000054,0.000000,0.000000,0.067451,0.000000,0.000335,0.000000,0.002981,0.000000,0.000000,0.000000,0.002077,0.000000,0.000394,9,0.200000,0.341667,0.607967,"Newt Gingrich: Pelosi’s grip slips as Ocasio-Cortez and friends throw a party, leave the grandparents at home https://t.co/UV5AAYz5iP #FoxNews For those who vote Dem, You better get use to this, Socialism now, Communism in the near future. Bureaucrats and Slaves #WalkAway"
150,1359616,SuperEliteTexan,68738,9.0,RT @Boyd_2650: 🇺🇸🇮🇱2)The bigger question is👉Is .@SpeakerPelosi lo

In [76]:
display_topics(nmf_9, tfidf_9.get_feature_names(), 10)


Topic  0
palestinian, israel, choose palestinian, faction hama, palestinian side, palestinian faction, tlaib grow, pay terrorist, side pay, number democrat

Topic  1
uh, mess bronx, borough perfect, though borough, without roasted, jh xatvtcm, people mess, woman without, clapback game, roasted act

Topic  2
class american, class, american, working class, working, work class, white, screw, screw work, pit white

Topic  3
condemn, anti semitism, semitism, anti, obscure anti, semitism ashamed, repeatedly help, design obscure, force democratic, condemn condemn

Topic  4
claim, crew postal, worker claim, box fraud, claim visit, represent neighbor, never boyfriend, postal worker, claim mail, film news

Topic  5
state america, united state, united, state, america, garbage great, firsthand united, family immigrate, tell firsthand, immigrate united

Topic  6
bigotry ilhanmn, resolution floor, put sham, floor design, design protect, outrage party, party put, semitic hate, hate bigotry, protect 

AOC: Climate change is topic 18
Bill Nye: Climate Change is topic 12



Average sentiment for each group, and then chart it

In [87]:
df_com_4.describe()
#Charlie Kirk

,index,followers_count,modularity_class,PageRank,rt_count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,doc_topic,polarity,subjectivity,pol_z
count,2.247930e+05,2.247930e+05,224793.0,2.247930e+05,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,224793.000000,2.247930e+05
mean,7.848175e+05,2.978425e+03,4.0,5.454545e-06,3458.635905,0.004026,0.003197,0.002943,0.002764,0.002470,0.002009,0.002036,0.002494,0.001774,0.002118,0.002062,0.001588,0.001711,0.001708,0.001574,0.002590,0.002451,0.001771,0.001715,0.001939,10.279083,0.013065,0.410985,4.226581e-15
std,4.900318e+05,4.246072e+04,0.0,1.087929e-04,4922.738670,0.019610,0.017517,0.016888,0.016442,0.015496,0.014697,0.013739,0.016315,0.010642,0.015535,0.016016,0.014268,0.014642,0.013442,0.011777,0.012581,0.012986,0.013096,0.013747,0.013034,6.080464,0.255431,0.279818,1.000002e+00
min,0.000000e+00,0.000000e+00,4.0,7.359514e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-9.120213e+00
25%,3.468150e+05,1.170000e+02,4.0,1.131959e-06,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,-0.104167,0.200000,-4.085651e-01
50%,7.525540e+05,3.890000e+02,4.0,2.178487e-06,1049.000000,0.000000,0.000000,0.000152,0.000000,0.000000,0.000016,0.000000,0.000035,0.000030,0.000091,0.000021,0.000000,0.000000,0.000089,0.000027,0.000400,0.000245,0.000124,0.000000,0.000158,10.000000,0.000000,0.450000,-6.913003e-02
75%,1.239935e+06,1.680000e+03,4.0,4.333874e-06,5309.000000,0.000000,0.000033,0.000528,0.000000,0.000049,0.000506,0.000021,0.000649,0.000188,0.000535,0.000357,0.000003,0.000032,0.000552,0.000385,0.001117,0.001529,0.000604,0.000117,0.000495,16.000000,0.117361,0.582500,4.533293e-01
max,1.604944e+06,1.418358e+07,4.0,1.949873e-02,49555.000000,0.105189,0.117060,0.122487,0.124018,0.124106,0.129085,0.126103,0.150312,0.107273,0.155847,0.159158,0.152446,0.159735,0.149683,0.135610,0.144880,0.157714,0.157554,0.168183,0.163633,19.000000,1.000000,1.000000,8.388820e+00


In [88]:
df_com_5.describe()
#moderate repubs

,index,followers_count,modularity_class,PageRank,rt_count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,doc_topic,polarity,subjectivity,pol_z
count,2.377460e+05,2.377460e+05,237746.0,2.377460e+05,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,237746.000000,2.377460e+05
mean,8.025199e+05,3.278231e+03,5.0,1.113962e-05,2204.467924,0.001585,0.002520,0.002410,0.001698,0.001746,0.001662,0.002116,0.001762,0.001991,0.001853,0.002436,0.001823,0.001791,0.002135,0.001622,0.001787,0.001734,0.001365,0.001845,0.003451,11.516215,0.028509,0.395988,-1.929332e-16
std,4.712861e+05,4.889663e+04,0.0,2.392397e-04,3650.212776,0.009826,0.016843,0.014206,0.011114,0.013064,0.011200,0.014830,0.012049,0.014906,0.013939,0.013496,0.012408,0.013641,0.013144,0.010429,0.012067,0.012555,0.012647,0.012977,0.012136,6.133479,0.272930,0.287728,1.000002e+00
min,1.000000e+00,0.000000e+00,5.0,7.309827e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-6.186133e+00
25%,3.799668e+05,1.150000e+02,5.0,1.287514e-06,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000011,6.000000,-0.083333,0.145000,-4.275584e-01
50%,8.193210e+05,3.970000e+02,5.0,2.446823e-06,417.000000,0.000005,0.000120,0.000006,0.000012,0.000000,0.000000,0.000034,0.000049,0.000089,0.000007,0.000033,0.000000,0.000118,0.000248,0.000029,0.000016,0.000049,0.000058,0.000000,0.000852,12.000000,0.000000,0.400000,-9.325307e-02
75%,1.204794e+06,1.592000e+03,5.0,5.372164e-06,2992.000000,0.000481,0.000359,0.000310,0.000208,0.000004,0.000077,0.000348,0.000346,0.000398,0.000113,0.000384,0.000062,0.000464,0.000796,0.000335,0.000259,0.000646,0.000566,0.000078,0.002118,18.000000,0.142857,0.583333,4.331598e-01
max,1.604948e+06,1.646032e+07,5.0,1.317517e-02,32788.000000,0.085309,0.144324,0.127777,0.108286,0.116073,0.119026,0.145361,0.126378,0.159770,0.151195,0.134684,0.138177,0.156880,0.156400,0.127690,0.147753,0.150467,0.163047,0.163085,0.122128,19.000000,1.000000,1.000000,5.694342e+00


In [91]:
df_com_9.describe()
#fox news

,index,followers_count,modularity_class,PageRank,rt_count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,doc_topic,polarity,subjectivity,pol_z
count,4.253430e+05,4.253430e+05,425343.0,4.253430e+05,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,425343.000000,4.253430e+05
mean,8.045112e+05,8.025562e+03,9.0,1.469310e-05,1710.244252,0.001661,0.001418,0.001126,0.001678,0.001548,0.001481,0.002353,0.001356,0.001137,0.001354,0.001422,0.001834,0.001291,0.001663,0.000877,0.002411,0.001149,0.003219,0.001550,0.002715,11.638689,0.013247,0.409104,-2.132684e-15
std,4.700102e+05,2.166502e+04,0.0,6.752441e-05,3288.039012,0.012590,0.009896,0.005862,0.011607,0.011454,0.010981,0.012169,0.010880,0.005143,0.010649,0.009520,0.011394,0.010751,0.011386,0.008948,0.009517,0.010608,0.008577,0.009414,0.011164,6.056787,0.272719,0.292684,1.000001e+00
min,2.000000e+00,0.000000e+00,9.0,7.165224e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-5.425278e+00
25%,3.993725e+05,8.530000e+02,9.0,2.533082e-06,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000331,0.000000,0.000006,6.000000,-0.111111,0.150000,-3.994240e-01
50%,8.066180e+05,3.064000e+03,9.0,5.314334e-06,293.000000,0.000000,0.000000,0.000000,0.000179,0.000088,0.000000,0.000357,0.000026,0.000076,0.000154,0.000112,0.000215,0.000000,0.000130,0.000155,0.000264,0.000000,0.001154,0.000216,0.000511,13.000000,0.000000,0.428571,-5.110756e-02
75%,1.222595e+06,7.807000e+03,9.0,1.015275e-05,1651.000000,0.000003,0.000100,0.000233,0.000463,0.000704,0.000201,0.000943,0.000254,0.000815,0.000365,0.000425,0.000624,0.000052,0.000460,0.000329,0.001165,0.000186,0.002589,0.000618,0.001260,17.000000,0.133333,0.600000,4.174390e-01
max,1.604943e+06,3.984765e+06,9.0,3.480735e-03,32787.000000,0.119354,0.113816,0.070209,0.141185,0.145862,0.142785,0.143025,0.140661,0.073287,0.146124,0.104812,0.153197,0.154066,0.164427,0.132581,0.103274,0.156194,0.094461,0.129595,0.160009,19.000000,1.000000,1.000000,5.354453e+00


In [89]:
df_com_6.describe()
#aoc

,index,followers_count,modularity_class,PageRank,rt_count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,doc_topic,polarity,subjectivity,pol_z
count,1.909980e+05,1.909980e+05,190998.0,1.909980e+05,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,190998.000000,1.909980e+05
mean,8.857038e+05,7.746995e+03,6.0,1.584706e-05,3066.087676,0.006004,0.005768,0.002918,0.002193,0.002192,0.002257,0.001790,0.002053,0.001612,0.002173,0.001842,0.001902,0.002503,0.001554,0.002033,0.001539,0.001684,0.001559,0.002394,0.001675,10.073859,0.082611,0.379412,-3.065343e-14
std,4.418958e+05,1.499684e+05,0.0,5.139383e-04,5817.809943,0.022841,0.022489,0.017781,0.015266,0.016086,0.016548,0.014794,0.014403,0.013701,0.014982,0.014341,0.014009,0.014016,0.013537,0.012404,0.013593,0.013234,0.012827,0.013297,0.012947,6.125124,0.254622,0.322965,1.000003e+00
min,4.500000e+01,0.000000e+00,6.0,8.503795e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-7.986643e+00
25%,5.282198e+05,1.450000e+02,6.0,1.164024e-06,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,-3.973398e-01
50%,9.539595e+05,4.920000e+02,6.0,2.237795e-06,296.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000015,0.000025,0.000011,0.000036,0.000040,0.000052,0.000128,0.000028,0.000102,0.000049,0.000110,0.000000,0.000211,0.000062,11.000000,0.000000,0.400000,-7.798318e-02
75%,1.257825e+06,1.726000e+03,6.0,4.343912e-06,2773.000000,0.000284,0.000306,0.000013,0.000026,0.000017,0.000031,0.000295,0.000481,0.000375,0.000313,0.000450,0.000213,0.000820,0.000416,0.000349,0.000159,0.000535,0.000132,0.000789,0.000638,16.000000,0.233333,0.625000,3.795985e-01
max,1.604947e+06,1.550397e+07,6.0,2.856027e-02,159517.000000,0.096186,0.099085,0.126751,0.132395,0.142311,0.153740,0.149854,0.151157,0.147202,0.162473,0.161606,0.162246,0.165636,0.167628,0.146862,0.178213,0.175045,0.171392,0.157023,0.178066,19.000000,1.000000,1.000000,7.801681e+00


In [90]:
df_com_7.describe()
#bill nye etc

,index,followers_count,modularity_class,PageRank,rt_count,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,doc_topic,polarity,subjectivity,pol_z
count,1.356120e+05,1.356120e+05,135612.0,1.356120e+05,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,135612.000000,1.356120e+05
mean,7.355190e+05,4.170083e+03,7.0,9.476694e-06,3876.475983,0.007683,0.006517,0.006774,0.003420,0.002092,0.003103,0.003339,0.002155,0.002659,0.002435,0.002378,0.002454,0.002736,0.001960,0.002366,0.001819,0.001807,0.002509,0.001542,0.001297,9.471536,0.098704,0.413714,8.700994e-15
std,4.175810e+05,1.767010e+05,0.0,1.993430e-04,5688.881476,0.025912,0.024363,0.025977,0.019705,0.014057,0.021271,0.021518,0.014391,0.019353,0.018052,0.017508,0.017900,0.019510,0.015800,0.015836,0.013709,0.016102,0.014091,0.014136,0.014023,6.287916,0.226706,0.286983,1.000004e+00
min,8.000000e+00,0.000000e+00,7.0,9.363664e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.568466e+01
25%,4.197785e+05,1.050000e+02,7.0,1.194706e-06,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.150000,-3.624577e-01
50%,7.001950e+05,3.360000e+02,7.0,1.783621e-06,505.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000010,0.000033,0.000037,0.000000,0.000271,0.000010,0.000000,10.000000,0.077778,0.500000,1.225878e-02
75%,1.046200e+06,1.126000e+03,7.0,3.501305e-06,4518.000000,0.000214,0.000229,0.000143,0.000131,0.000278,0.000194,0.000386,0.000215,0.000011,0.000206,0.000252,0.000000,0.000246,0.000265,0.000363,0.000402,0.000069,0.001398,0.000433,0.000041,16.000000,0.200000,0.550649,4.342791e-01
max,1.604921e+06,4.150207e+07,7.0,1.515572e-02,159517.000000,0.098295,0.101987,0.108683,0.133107,0.110050,0.167070,0.168822,0.122630,0.159590,0.150855,0.152904,0.159421,0.173579,0.165930,0.167691,0.162877,0.192502,0.181402,0.193448,0.192022,19.000000,1.000000,1.000000,1.656203e+01
